In this file, we generate code to generate samples, then compute correlations (nearest neighbor, next nearest neighbor and average spin) and then learn directly from correlations only.

In [ ]:
# ising_moment_rg.jl

using Random
using Statistics
using Optim

# ----------------------------------------------------------------------------
# 1) Define the 1D Ising model and MCMC sampler
# ----------------------------------------------------------------------------

struct IsingModel1D
    lattice_size::Int
    J::Float64
    h::Float64
end

function total_energy(spins::Vector{Int8}, model::IsingModel1D)
    N = model.lattice_size
    E = 0.0
    @inbounds for i in 1:N
        ip = (i == N ? 1 : i+1)
        E -= model.J * spins[i] * spins[ip]
        E -= model.h * spins[i]
    end
    return E
end

#Generate mcmc samples using Metropolis-Hastings
function mcmc_samples(model::IsingModel1D, T::Float64, n_steps::Int)
    N = model.lattice_size
    spins = Int8.(rand([-1, 1], N))
    current_energy = total_energy(spins, model)
    samples = Vector{Vector{Int8}}(undef, n_steps)
    for step in 1:n_steps
        i = rand(1:N)
        spins[i] = -spins[i]
        new_energy = total_energy(spins, model)
        ΔE = new_energy - current_energy
        if ΔE <= 0 || rand() < exp(-ΔE/T)
            current_energy = new_energy
        else
            spins[i] = -spins[i]  # reject
        end
        samples[step] = copy(spins)
    end
    return samples
end

# ----------------------------------------------------------------------------
# 2) Compute empirical moments (m, c1, c2)
# ----------------------------------------------------------------------------
# Compute correlators m = <s_i>, c1 = <s_i s_{i+1}>, c2 = <s_i s_{i+2}>
function compute_moments(samples::Vector{Vector{Int8}})
    n_conf = length(samples)
    N      = length(samples[1])
    sum_s   = 0.0
    sum_ss1 = 0.0
    sum_ss2 = 0.0

    for s in samples
        sum_s += sum(s)
        for i in 1:N
            sum_ss1 += s[i] * s[mod1(i+1, N)]
            sum_ss2 += s[i] * s[mod1(i+2, N)]
        end
    end

    denom = n_conf * N
    m  = sum_s   / denom
    c1 = sum_ss1 / denom
    c2 = sum_ss2 / denom
    return m, c1, c2
end

# ----------------------------------------------------------------------------
# 3) Define the moment‐based loss function L(J,h)
# ----------------------------------------------------------------------------
function make_moment_loss(m, c1, c2)
    # returns a function L(θ) where θ = [J, h]
    return function (θ)
        J, h = θ
        Pp2 = (1 + 2*m + c2)/4
        P0  = (1 -      c2   )/2
        Pm2 = (1 - 2*m + c2)/4

        L = -h*m - 2J*c1 +
            Pp2*log(2*cosh(h+2J)) +
             P0*log(2*cosh(h   )) +
            Pm2*log(2*cosh(h-2J))
        return L
    end
end

# ----------------------------------------------------------------------------
# 4) Main script
# ----------------------------------------------------------------------------


Random.seed!(1234)

# 4.1 Generate MCMC samples
true_model = IsingModel1D(10, 0.35, 0.15)
println("Running MCMC on 1D Ising with J=$(true_model.J), h=$(true_model.h)...")
samples = mcmc_samples(true_model, 1.0, 500_000)

# 4.2 Compute empirical moments
m, c1, c2 = compute_moments(samples)
println("Empirical moments: m = $m, c1 = $c1, c2 = $c2")

# 4.3 Build loss and optimize (J,h)
loss = make_moment_loss(m, c1, c2)
θ0 = [0.5, 0.0]  # initial guess for [J, h]
result = optimize(loss, θ0, BFGS(); autodiff = :forward)
J_fit, h_fit = Optim.minimizer(result)

println("\nFitted parameters from moments:")
println("J_fit = $(round(J_fit, digits=4)), h_fit = $(round(h_fit, digits=4))")


Now we generate samples of various sizes and check if errors scale correctly as $1/\sqrt{N}$

In [ ]:
N       = 4
true_J  = 0.35
true_h  = 0.15
model   = IsingModel1D(N, true_J, true_h)

T             = 1.0
repetitions   = 100
sample_sizes  = [Int(round(4000*1.5^i)) for i in 3:15]

# prepare storage
errors_J = Dict(s => Float64[] for s in sample_sizes)
errors_h = Dict(s => Float64[] for s in sample_sizes)

for S in sample_sizes
    println("Running sample_size = $S")
    for rep in 1:repetitions
        # 1) generate S Monte Carlo draws
        samples = mcmc_samples(model, T, S)

        # 2) compute moments and build loss
        m, c1, c2   = compute_moments(samples)
        loss        = make_moment_loss(m, c1, c2)
        θ0          = [true_J, true_h]
        result      = optimize(loss, θ0, BFGS(); autodiff = :forward)
        J_fit, h_fit = Optim.minimizer(result)

        # 3) record absolute errors
        push!(errors_J[S], abs(J_fit - true_J))
        push!(errors_h[S], abs(h_fit - true_h))
    end
end
